In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
import torch
from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch

2023-01-19 17:02:37.125392: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 17:02:38.093813: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-19 17:02:38.093887: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-19 17:02:38.093897: W tensorfl

In [3]:
device = torch.device("cuda")#ppo_trainer.accelerator.device

In [4]:
pretrained = ""#"flax-community/gpt2-base-thai"

In [5]:
# get models
model = AutoModelForCausalLMWithValueHead.from_pretrained(pretrained).to(device)
model_ref = create_reference_model(model).to("cuda")

In [6]:
tokenizer = AutoTokenizer.from_pretrained(pretrained)

In [7]:
# initialize trainer
ppo_config = PPOConfig(
    batch_size=1,
    forward_batch_size=1
)
ppo_trainer = PPOTrainer(ppo_config, model, model_ref, tokenizer)

/workspace/gpt2moon/trl/trl/trainer/ppo_trainer.py:137: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [8]:
# # encode a query
# query_txt = "Q:อธิบายข้อเสียของสภาวะโลกร้อนให้ฟังหน่อย \nA:"
# query_tensor = tokenizer.encode(query_txt, return_tensors="pt")

# # get model response
# response_tensor  = respond_to_batch(model_ref, query_tensor)

In [9]:
query_txt = "Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:"
while True:
    query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
    response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
    print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")
    _i = input("reward: ")
    reward = [torch.tensor(float(_i))]
    train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
    if _i=="1":
        query_txt+=str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>"
        print(query_txt)
        add = input("Q: ")
        if add=="exit":
            break
        query_txt+=" Q:"+add+"</s> A:"

Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:คุณหมออยากรู้เลยค่ะว่าจะน่าเอาเป็นผู้ป่วยมะเร็งไหมค่ะ</s>
reward: 00
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:ตรงนี้เราก็เป็นคนหนึ่งอยู่แล้ว</s>
reward: 00
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:ผมเป็นคนปากีสถานครับ ข้อมูลค่อนข้างน้อยเลย จะเรียนวิทยาศาสตร์ไปก็ไม่ได้ไปสอบ สุดท้ายก็จบโทไทย สูงประมาณ 160 เลขใบขับขี่ ไม่ใช่ไทยนะครับ ภาษาลาวเป็นหนามและแขน 555</s>
reward: 00
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:ให้ลองเป็นนักเรียนนวด thai body best หรือ Mind body best กันนะคะ</s>
reward: 00
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:สวัสดีค่ะ ชื่อหนูแพรครับ</s>
reward: 11
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:สวัสดีค่ะ ชื่อหนูแพรครับ</s>
Q: ยินดีต้อนรับครับคุณแพร คุณทำอะไรอยู่ครับยินดีต้อนรับครับคุณแพร คุณทำอะไรอยู่ครับ
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:สวัสดีค่ะ ชื่

reward: 00
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:สวัสดีค่ะ ชื่อหนูแพรครับ</s> Q:ยินดีต้อนรับครับคุณแพร คุณทำอะไรอยู่ครับ</s> A:ขอบคุณครับ อยู่บ้านเล่นโทรศัพท์มากกว่าค่ะ</s> Q:เล่นเกมอะไรอ่ะครับ</s> A:ถึงเป็นเกมนึงนะครับ(จริง ๆ ก็เงียบไม่มาก) คือเล่นเกมส์ติดต่อเพื่อดูภาพยนตร์เรื่องนวนิยายที่ขึ้นปกนิตยสารเเล้วแง่กิจกรรมต่างๆ(จริงๆ ก็แอบเเจะครับ)</s> Q:น่าสนุกนะครับ แล้วคุณกินอะไรยังครับ</s> A:ได้รวมเก๊กฮวยเก๊กเฮลกอยู่ที่ร้านนองรักด้วยกัน</s>
reward: 00
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:สวัสดีค่ะ ชื่อหนูแพรครับ</s> Q:ยินดีต้อนรับครับคุณแพร คุณทำอะไรอยู่ครับ</s> A:ขอบคุณครับ อยู่บ้านเล่นโทรศัพท์มากกว่าค่ะ</s> Q:เล่นเกมอะไรอ่ะครับ</s> A:ถึงเป็นเกมนึงนะครับ(จริง ๆ ก็เงียบไม่มาก) คือเล่นเกมส์ติดต่อเพื่อดูภาพยนตร์เรื่องนวนิยายที่ขึ้นปกนิตยสารเเล้วแง่กิจกรรมต่างๆ(จริงๆ ก็แอบเเจะครับ)</s> Q:น่าสนุกนะครับ แล้วคุณกินอะไรยังครับ</s> A:อืมส์ดื่มน้ําหรือครับ</s>
reward: 00
Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A:สวั

In [10]:
query_txt="Q:พรุ่งนี้อากาศจะเป็นอย่างไรบ้าง</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100)
print(query_txt+"__"+str(tokenizer.decode(response_tensor[0])))

Q:พรุ่งนี้อากาศจะเป็นอย่างไรบ้าง</s> A:__ภาคเหนือสัปดาห์หน้ายังมีอากาศร้อน สําหรับสัปดาห์ที่แล้วอากาศร้อนแม้ไม่ได้ลดในวันนี้ แต่ถูกขับคาตอลิ่งมากกว่าเมื่อวานด้วยฝนลงมา ซึ่งสภาพลมระหว่างระหว่างรัฐบัลลังก์และปูร์ได้กลับมากทม. ส่วนคลื่นลมบริเวณนั้น นักพยากรณ์เนย์มาร์ คาดว่ามีแนวโน้มว่าจะซัดบ้านเรือนหาดเจ้าหลาว ชาวกระบี่ หลังฝนตกติดต่อกันหลายชั่วโมง</s> Q:ในสัปดาห์นี้ 1.วันที่ 9


In [11]:
response_tensor[0]

tensor([     6,  20850,  45475, 121165,   6065,  47727,  37549,  39045,  21309,
        121165, 114195,  37549,  39045,  69576,  13901,  19308,   1274,  41274,
          4034,  12783,  66063,  20529,   4154,  23450,  80683,  31507,  12215,
        143886,   4470,  16979,   1485,   6556,   2293,   8193,  45567,  31672,
         35574,  35574,  37642,   2469,  45399, 108117,   2014,   4310,   1347,
         74735,  15317,   1621,  46071,   2014, 131768,      5,  29020,   2107,
        179938,  31672,  50440,   5812,  59843, 250094, 121527,  24227,  14479,
        103585,      6,  57669, 139419,  28255,  20393,  19363,  52521,   5186,
         13674,   8017,  99455,  69952,  17587, 149808,   3725,      6,  23861,
        138690,  27124,  68715,  16979,   1485,  24717,  57584,   3503,  15970,
        236460,      2,   1336,     13,   1274, 121165,   2417,    588,  39090,
           465], device='cuda:0')

In [12]:
tokenizer.encode("</s>")

[2, 2]

In [13]:
"Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนำตัวเองได้ไหม</s> A: </s> Q:สวัสดีครับ</s> A:สวัสดีค่ะ</s> Q:คุณช่วยแนะนําตัวเองได้ไหม</s> A:Sale จ้า เป็นสาวกใหม่แต่ตัวเองค่อนข้างเป็นคนขี้อาย</s>".split("</s>")[-2]

' A:Sale จ้า เป็นสาวกใหม่แต่ตัวเองค่อนข้างเป็นคนขี้อาย'

In [27]:
query_txt="Q:พรุ่งนี้อากาศจะเป็นอย่างไรบ้าง</s> A:ภาคเหนือสัปดาห์หน้ายังมีอากาศร้อน สําหรับสัปดาห์ที่แล้วอากาศร้อนแม้ไม่ได้ลดในวันนี้ แต่ถูกขับคาตอลิ่งมากกว่าเมื่อวานด้วยฝนลงมา ซึ่งสภาพลมระหว่างระหว่างรัฐบัลลังก์และปูร์ได้กลับมากทม. ส่วนคลื่นลมบริเวณนั้น นักพยากรณ์เนย์มาร์ คาดว่ามีแนวโน้มว่าจะซัดบ้านเรือนหาดเจ้าหลาว ชาวกระบี่ หลังฝนตกติดต่อกันหลายชั่วโมง</s> Q:แล้วที่กรุงเทพล่ะ</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:พรุ่งนี้อากาศจะเป็นอย่างไรบ้าง</s> A:ภาคเหนือสัปดาห์หน้ายังมีอากาศร้อน สําหรับสัปดาห์ที่แล้วอากาศร้อนแม้ไม่ได้ลดในวันนี้ แต่ถูกขับคาตอลิ่งมากกว่าเมื่อวานด้วยฝนลงมา ซึ่งสภาพลมระหว่างระหว่างรัฐบัลลังก์และปูร์ได้กลับมากทม. ส่วนคลื่นลมบริเวณนั้น นักพยากรณ์เนย์มาร์ คาดว่ามีแนวโน้มว่าจะซัดบ้านเรือนหาดเจ้าหลาว ชาวกระบี่ หลังฝนตกติดต่อกันหลายชั่วโมง</s> Q:แล้วที่กรุงเทพล่ะ</s> A:กรุงเทพจะเป็นจังหวัดที่ร้อนที่สุดเมื่อเทียบกับภาคอื่น ตอนนี้จะเข้าสู่ช่วงที่การหิมะออกเยอะ สภาพอากาศไม่ค่อยได้เลยสําหรับบางพื้นที่ ขณะที่ระดับน้ําจะทรงตัว และอึมครึม</s>


In [18]:
tokenizer.decode(response_tensor[0]).split("</s>")[-2].replace("A:","").strip()+"</s>"

'ฝนตกมากที่สุดตั้งแต่30-18.00น. มีฝนลมแรงตกทุกแห่งทั่วทั้งกรุงเทพ โดยเฉพาะเหนือ-ใต้ฝั่งอ่าวไทยเล็กน้อยและฝนตกหนักให้ตกทั่วทั้งจังหวัด โดยเฉพาะวันนี้ที่อ่าวไทยมีฝนตกหนักติดต่อกันเกือบทั้งวัน ภาคใต้ฝั่งตะวันออกมีฝนตกเกือบทั้งวัน</s>'

In [26]:
str(tokenizer.decode(response_tensor[0]))

'อย่างที่รู้ดีว่า อัทธยานิยมเกิดในปี พ.ศ.2548 ร้อนอย่างรุนแรงฤดูร้อน พ.ศ.2550 มาปีนี้ทางบริติช เคานซิล จะได้รับผลกระทบไปด้วย ดังนั้นสิงหาคมปีหน้าอากาศจะเป็นแบบนี้แหละ บ้านเรือน คุณยายก็คงอ่วมไปทั่วประมาณ 3 วันแล้ว การใช้ชีวิตต้องปรับตัวกันทั้งนั้น คุณอาจเจ็บป่วย หรือแม่เขาขาขาวได้ อย่าเอาร่างกายเป็นศูนย์กลางกายใจ ขับรถเป็นระเบียบ อย่าละเมิดกฎ '

In [32]:
query_txt = "Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:"
while True:
    query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
    response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
    while True:
        response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
        ptext = str(tokenizer.decode(response_tensor[0]))
        if len(ptext.split("</s>"))>1:
            break
    print(query_txt+ptext.split("</s>")[-2].replace("A:","").strip()+"</s>")
    _i = input("reward: ")
    reward = [torch.tensor(float(_i))]
    train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
    if _i=="1":
        query_txt+=ptext.split("</s>")[-2].replace("A:","").strip()+"</s>"
        print(query_txt)
        add = input("Q: ")
        if add=="exit":
            break
        query_txt+=" Q:"+add+"</s> A:"

Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:เป็นสิ่งส่งผลกระทบต่อโลกของเราโดยตรง เพราะฉะนั้นการที่เราละเลย ไม่ได้เข้าร่วมกับการส่งเสริมการเลี้ยงดู 'แม่'ขึ้นมาใหม่นี่แหละค่ะ</s>
reward: 00
Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:หากปล่อยคือทางตัน แต่ไม่มีหลักฐานอ้างอิง กรณีสึนามิ นักเรียนภาคหนักของนักกีฬาพนักงานและกรรมการผู้ตัดสินจะมีอายุสั้นกว่าวัยทํางานทั่วไป 1.5 ปี</s>
reward: 00
Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:สภาวะโลกร้อนในปัจจุบัน มีสาเหตุจากหลากหลายสาเหตุ รวมทั้งการลดลงของระดับน้ําในแม่น้ําที่ไหลผ่านจึงสลายตัวเป็นน้ําแข็ง นอกจากนี้ยังส่งผลให้สังเกตพบว่าอุณหภูมิในบางระดับเปลี่ยนแปลงอย่างรวดเร็ว</s>
reward: 00
Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:สภาวะธรรมชาติถูกสร้างขึ้นมาจากอนุมูลอิสระที่ย่อยสลายได้เองและประกอบกับปิโตรเลียมน้ํามัน มันจึงถูกดึงโดยเร่ธาตุที่ทําให้เกิดการสึกกร่อนออกไซด์อย่างรวดเร็ว นั่นก็คือ ร้อน (ร้อนกระทําความร้อนให้ความร้อนแก่สิ่งมีชีวิต) มันจึงเ

reward: 00
Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:สภาวะโลกร้อนเกิดจากปรากฏการณ์ทางธรรมชาติที่เกิดจากกิจกรรมของมนุษย์ ซึ่งเกิดจากกิจกรรมต่างๆในชีวิตประจําวัน เช่น การทําเหมืองแร่ การเพาะเห็ด แกลบ แกง และกิจกรรมอื่นของภาคเกษตรกรรม เช่น การทําเกษตรอินทรีย์ ไถนา ปลูกพืช และการคมนาคม</s> Q:แล้วเราจะช่วยกันลดโลกร้อนได้อย่างไรบ้าง?</s> A:ช่วยกันปลูกต้นไม้ ส่งเสริมการใช้พลังงานพอเพียง และวิธีใช้พลังงานที่เหลือน้อยอย่างมีประสิทธิภาพ ลดการใช้พลังงานในระดับครัวเรือน ใช้ในอาคารบ้านเรือน ลดการใช้พลาสติก</s> Q:จริงด้วย เดี่ยวเราไปช่วยกันลดโลกร้อนกันนะ</s> A:ใช่ครับ เราพยายามลดการใช้พลังงานอย่างจริงจัง การแปรรูปพืช เป็นสัญลักษณ์ของความยั่งยืน ซึ่งโดยทั่วไปแล้วเกษตรกรส่งออกจะมีเครื่องนุ่งห่ม ผ้าขนหนู เครื่องปั๊มน้ํา และไฟฟ้าจากภาคเอกชนทั้งหมด</s>
reward: 00
Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:สภาวะโลกร้อนเกิดจากปรากฏการณ์ทางธรรมชาติที่เกิดจากกิจกรรมของมนุษย์ ซึ่งเกิดจากกิจกรรมต่างๆในชีวิตประจําวัน เช่น การทําเหมืองแร่ การเพาะเห็ด แกลบ แกง แล

Q: ขอบคุณครับขอบคุณครับ
Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:สภาวะโลกร้อนเกิดจากปรากฏการณ์ทางธรรมชาติที่เกิดจากกิจกรรมของมนุษย์ ซึ่งเกิดจากกิจกรรมต่างๆในชีวิตประจําวัน เช่น การทําเหมืองแร่ การเพาะเห็ด แกลบ แกง และกิจกรรมอื่นของภาคเกษตรกรรม เช่น การทําเกษตรอินทรีย์ ไถนา ปลูกพืช และการคมนาคม</s> Q:แล้วเราจะช่วยกันลดโลกร้อนได้อย่างไรบ้าง?</s> A:ช่วยกันปลูกต้นไม้ ส่งเสริมการใช้พลังงานพอเพียง และวิธีใช้พลังงานที่เหลือน้อยอย่างมีประสิทธิภาพ ลดการใช้พลังงานในระดับครัวเรือน ใช้ในอาคารบ้านเรือน ลดการใช้พลาสติก</s> Q:จริงด้วย เดี่ยวเราไปช่วยกันลดโลกร้อนกันนะ</s> A:ทําตามขั้นตอนที่เราเขียนไว้ข้างต้นก็ได้นะครับ</s> Q:ขอบคุณครับ</s> A:อาจจะจะเป็น โรงเรียน กวดวิชา มาร่วมสมัครกันด้วยนะครับ วันละ 8 คน วันนี้เป็นวันสุดท้ายของโครงการ ขอบคุณล่วงหน้าครับ</s>
reward: 00
Q:สวัสดี ถามอะไรหน่อย ช่วยอธิบายสาเหตุของสภาวะโลกร้อนให้หน่อย</s> A:สภาวะโลกร้อนเกิดจากปรากฏการณ์ทางธรรมชาติที่เกิดจากกิจกรรมของมนุษย์ ซึ่งเกิดจากกิจกรรมต่างๆในชีวิตประจําวัน เช่น การทําเหมืองแร่ การเพาะเห็ด แ

In [35]:
query_txt="Q:พรุ่งนี้อากาศจะเป็นอย่างไรบ้าง</s> A:ภาคเหนือสัปดาห์หน้ายังมีอากาศร้อน สําหรับสัปดาห์ที่แล้วอากาศร้อนแม้ไม่ได้ลดในวันนี้ แต่ถูกขับคาตอลิ่งมากกว่าเมื่อวานด้วยฝนลงมา ซึ่งสภาพลมระหว่างระหว่างรัฐบัลลังก์และปูร์ได้กลับมากทม. ส่วนคลื่นลมบริเวณนั้น นักพยากรณ์เนย์มาร์ คาดว่ามีแนวโน้มว่าจะซัดบ้านเรือนหาดเจ้าหลาว ชาวกระบี่ หลังฝนตกติดต่อกันหลายชั่วโมง</s> Q:แล้วที่กรุงเทพล่ะ</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:พรุ่งนี้อากาศจะเป็นอย่างไรบ้าง</s> A:ภาคเหนือสัปดาห์หน้ายังมีอากาศร้อน สําหรับสัปดาห์ที่แล้วอากาศร้อนแม้ไม่ได้ลดในวันนี้ แต่ถูกขับคาตอลิ่งมากกว่าเมื่อวานด้วยฝนลงมา ซึ่งสภาพลมระหว่างระหว่างรัฐบัลลังก์และปูร์ได้กลับมากทม. ส่วนคลื่นลมบริเวณนั้น นักพยากรณ์เนย์มาร์ คาดว่ามีแนวโน้มว่าจะซัดบ้านเรือนหาดเจ้าหลาว ชาวกระบี่ หลังฝนตกติดต่อกันหลายชั่วโมง</s> Q:แล้วที่กรุงเทพล่ะ</s> A:ขณะนี้สภาพอากาศในกรุงเทพฯ อยู่ในเทรนด์ที่ดี ส่วนใหญ่ยังคงลมเย็น ลมแรง ส่วนภาคใต้มีอากาศร้อนจัด ติดตาม 5 พ.ค. ที่นี่</s>


In [37]:
query_txt="Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างำร</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างำร</s> A:การไม่กล่าว วันนี้จะทําให้คุณไม่ได้ทําให้ใครผิดหวัง รักคุณเท่าที่คุณทําได้</s>


In [41]:
query_txt="Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = responaad_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:หากคุณแอบรัก หรือทําตัวเจ้าชู้ อาศัยเรื่องของใจเป็นตัวบ่งบอกว่าคุณยังเป็นแฟนเขา ไม่จําเป็นต้องปรึกษาฝ่ายหญิง หรืออะไรทุกอย่างให้ชัดเจนนะ เธอคิดแน่นอน และแน่นอนอยู่ๆก็ขาดใจเด็ดขาด ถ้าคุณยังทําใจไม่ได้ ก็ดูใจเธออย่างจริงจัง ทางที่ดีนัดกันไปก็แล้วกัน แล้วเธอก็จะเข้าใจและเข้าใจ คุณไม่ต้องมาคันปาก</s>


In [43]:
reward = [torch.tensor(0.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [47]:
query_txt="Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:ขั้นแรกก็คือต้องตัวเองให้หญิงมาเป็นแฟน หญิงจะต้องอุ้มหลานไปก่อน แล้วค่อยจัดเวลาให้ได้ ถ้ากลัวหิวอาจจะพาไปทานเวลากลางคืน เป็นต้น</s>


In [48]:
reward = [torch.tensor(0.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [49]:
query_txt="Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:มันจะดูหวานล่อนิดๆ แต่อบอุ่นหน่อยๆ (เรื่องนี้เขียนโดยพี่นุช เลยไม่เหมือนหนู )</s>


In [50]:
reward = [torch.tensor(0.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [54]:
query_txt="Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:นอกจากตอนที่รู้จักกัน อย่าง รู้จักกันได้ทุกนาทีจากบทเรียนที่ยากมากๆของ มาร์ค ในช่วงๆ ที่มัดมือ</s>


In [55]:
reward = [torch.tensor(0.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [62]:
query_txt="Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:"
while True:
    query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
    response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
    while True:
        response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
        ptext = str(tokenizer.decode(response_tensor[0]))
        if len(ptext.split("</s>"))>1:
            break
    print(query_txt+ptext.split("</s>")[-2].replace("A:","").strip()+"</s>")
    _i = input("reward: ")
    reward = [torch.tensor(float(_i))]
    train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
    if _i=="1":
        query_txt+=ptext.split("</s>")[-2].replace("A:","").strip()+"</s>"
        print(query_txt)
        add = input("Q: ")
        if add=="exit":
            break
        query_txt+=" Q:"+add+"</s> A:"

Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:การที่เราจะแต่งงานก็จริง แต่ห้าม</s>
reward: 00
Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:50 ข้อ "บางครั้ง" ลองทําดู</s>
reward: 00
Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:วันแรกที่ได้เจอกันคุณจะรู้สึกอย่างไร (ไม่ว่าจะใกล้แค่ไหน)</s>
reward: 11
Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:วันแรกที่ได้เจอกันคุณจะรู้สึกอย่างไร (ไม่ว่าจะใกล้แค่ไหน)</s>
Q: คุณสึกใจเต้น หลง ทำตัวไม่ถูก ใจชื้น ชอบ แอบรักตั้งแต่เจอกันคุณสึกใจเต้น หลง ทำตัวไม่ถูก ใจชื้น ชอบ แอบรักตั้งแต่เจอกัน
Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:วันแรกที่ได้เจอกันคุณจะรู้สึกอย่างไร (ไม่ว่าจะใกล้แค่ไหน)</s> Q:คุณสึกใจเต้น หลง ทำตัวไม่ถูก ใจชื้น ชอบ แอบรักตั้งแต่เจอกัน</s> A:คุณเปนลมแอบซ่า ชอบแอบใจร้อนมากแนะนําการพร้อนไลน์ละกัน อย่าทนได้แบบพี่เค้าหรอก (ลองหาคนที่สดใส แอบใจร้อน เรื่องใจ แล้วจะอินสุดๆ)</s>
reward: 00
Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:วันแรกที่ได้เจอกันคุณจะรู้สึกอย่างไร (ไม่ว่าจะใกล้แค่ไหน)</s> Q:คุณสึกใจเต้น หลง ทำตัวไม่ถูก ใจชื้

KeyboardInterrupt: Interrupted by user

In [61]:
print(train_stats)

{'objective/kl': -30.161563873291016, 'objective/kl_dist': [tensor([ 1.3102e+00, -9.4143e-02, -2.2765e-01, -5.5613e-02,  5.8686e-04,
         3.7514e-01, -1.9476e-02, -3.0494e-01, -2.8878e-04,  1.5344e-04,
         7.3922e-03, -2.3897e-01, -2.6681e-01, -9.8923e-04, -2.0653e-02,
        -2.5510e-02, -1.6439e-02, -1.6802e-02, -7.7270e-03,  1.2084e-04,
        -7.3867e-03, -7.4793e-03,  1.8673e-01, -3.4639e-03, -1.9593e-02,
        -7.4680e-03,  1.0251e-05, -2.2086e-03,  7.2016e-03, -4.3339e+00,
        -7.7256e-01,  2.0225e-02,  2.9731e-01,  2.5217e-02,  1.7617e-02,
        -2.8029e-01,  3.6254e-03,  1.0826e-01, -2.2823e+00,  7.0902e-01,
         1.8185e-02, -6.5479e+00,  6.2843e-01, -7.6460e-01, -4.4929e-01,
        -7.1776e-01, -5.3027e-01, -5.4220e-02,  2.3718e-01, -8.5772e-01,
        -6.3090e-01, -2.0187e-01, -3.8445e-01,  2.3216e-01, -2.3829e-01,
        -3.7042e-01,  1.8201e-01, -5.5538e-01, -3.9912e-01,  4.4328e-01,
        -5.3475e-01, -1.5946e-01,  2.7745e-01,  3.2878e-02,  5.4

In [68]:
query_txt="Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:รู้ใช่ไหมว่า คนที่แอบรักจะต้อง รักซึ่งกันและกัน</s>


In [69]:
reward = [torch.tensor(1.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [248]:
# ปรับแต่ง traning
query_txt="Q:อยากบอกรักให้กับคนที่แอบรักต้องอย่างไร</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
ans=tokenizer.encode(query_txt+"ขึ้นอยู่กับคุณนะคะ</s>", return_tensors="pt").to(device)
reward = [torch.tensor(float(1))]
train_stats = ppo_trainer.step([query_tensor[0]], [ans[0]], reward)

In [71]:
query_txt="Q:สภาพอากาศพรุ่งนี้จะเป็นอย่างไรบ้าง</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:สภาพอากาศพรุ่งนี้จะเป็นอย่างไรบ้าง</s> A:พรุ่งนี้อากาศจะเย็นนะครับ ร้อนมาก โดยเฉพาะจังหวัดภูเก็ตและกระบี่ อุณหภูมิจะ 19-20 องศา เป็นระยะๆ จะร้อนไปถึงตอนใกล้สิ้นปีเลย เดี๋ยวปิดเทอมจะมาแปะภาพมาให้ดู</s>


In [137]:
query_txt="Q:ทำไมวัยทำงานสมัยนี้มันเหนื่อย?</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:ทำไมวัยทำงานสมัยนี้มันเหนื่อย?</s> A:เวลาอายุมากตั้งแต่สมัยเด็กก็มักจะรู้สึกว่ามันเหนื่อยเหมือนกับที่เพิ่งออกจากโรงเรียนและสุขภาพจิตไม่ค่อยดี ก็เกิดเป็นคําถามว่า</s>


In [138]:
reward = [torch.tensor(0.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [143]:
query_txt="Q:ทำไมวัยทำงานสมัยนี้มันเหนื่อย?</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:ทำไมวัยทำงานสมัยนี้มันเหนื่อย?</s> A:การทํางานในยุคดิจิทัลทําให้คนวัยทํางานทํางานเหนื่อยไม่ต่างกัน</s>


In [144]:
reward = [torch.tensor(1.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [ ]:
Q:ทำไมวัยทำงานสมัยนี้มันเหนื่อย?</s> A:ชั่วโมงที่คนวัยทํางานทํางานเนี่ย ผมว่ามันเทียบไม่ได้เรื่องของคุณภาพชีวิต เพราะว่ามันไม่ใช่การแข่งขันกันระหว่างเดือนกับเดือน เป็นแค่งานที่ถือเป็นความฝัน แต่ความจริงก็แค่ใจที่ต้องทํามันเท่านั้นครับ ไม่ใช่หน้าที่ บางคนแพ้ก็เป็นพวกขี้อ้อน บางคนถึงขั้นหัวฟูก็ได้</s>


In [128]:
query_txt="Q:9+1=?</s> A:="
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=40,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:9+1=?</s> A:== Orient View and Window GlowingLandscape</s>


In [129]:
reward = [torch.tensor(0.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [252]:
query_txt="Q:9+1 เท่ากับอะไร</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=40,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:9+1 เท่ากับอะไร</s> A:ความเร็วและแรงบิดที่เดินทางผ่านสนามจะถูกระบุไว้ด้วยกราฟ</s>


In [218]:
reward = [torch.tensor(0.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [250]:
query_txt="Q:ทำไมวัยทำงานสมัยนี้มันเหนื่อย?</s> A:"
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)
response_tensor  = respond_to_batch(model_ref, query_tensor, txt_len=100,end=2)
print(query_txt+str(tokenizer.decode(response_tensor[0])).split("</s>")[-2].replace("A:","").strip()+"</s>")

Q:ทำไมวัยทำงานสมัยนี้มันเหนื่อย?</s> A:สาเหตุเพราะรู้สึกว่าเราต้องใช้เวลาส่วนใหญ่ในระยะนี้กับการสร้างระบบความสัมพันธ์กับครอบครัว สังคม และผู้ใหญ่</s>


In [251]:
reward = [torch.tensor(1.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [246]:
response_tensor[0]

tensor([     2,   1336,     13, 108899,  48670,  25132,  39331,   2417,  10120,
        189964,     28,      2,     46,     13,  13956,   6003,   2653,  33960,
         12339,   7637,   2165,  98901,   1153,  26256, 193568,  16401,  73465,
         24326,  77505,   6721, 122444,   3650,   4034,  67747,   3991, 135715,
        201518,   7210,  77766,  26698,   1485,      6,  13033,      6,  90380,
         17878,   3503, 142879, 168892,   1153,  99410,      2],
       device='cuda:0')

In [247]:
ans[0]

tensor([     2,   1336,     13,  33102,  25135,  15463,  36803,  43266,  24817,
          2469,  15463,   9022,  56987,      2,     46,     13, 207184,   7481,
         38903,      2], device='cuda:0')

In [239]:
reward = [torch.tensor(0.0)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)